# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that I will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment

from azureml.core.dataset import Dataset

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy,\
                                     choice, randint, PrimaryMetricGoal, HyperDriveConfig


from azureml.widgets import RunDetails

## Dataset

Get data. In the cell below, I write code to access the data I will be using in this project.

In [2]:
ws = Workspace.from_config(path="/")

experiment_name = 'HyperTune'

experiment=Experiment(ws, experiment_name)

In [3]:
found = False
key = "Employee Attrition"
description_text = "IBM HR Analytics Employee Attrition & Performance"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        url = "https://raw.githubusercontent.com/eljandoubi/Azure-Machine-Learning-Engineer/main/attrition-dataset.csv"
        dataset = Dataset.Tabular.from_delimited_files(url)        
        dataset = dataset.register(workspace=ws,
                                   name=key,description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,...,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,1.0,1024.865306,2.721769,65.891156,2.729932,2.063946,...,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,1.024165,0.0,602.024335,1.093082,20.329428,0.711561,1.106940,...,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,66.000000,3.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1.0,1555.750000,4.000000,83.750000,3.000000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,5.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [4]:
cluster_name = "automl-vs-hpyer"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target, using it!')
    
except ComputeTargetException:
    print('Creating a new compute target!')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',min_nodes=1, max_nodes=8)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)
 
cpu_cluster.get_status().serialize()

Found existing compute target, using it!
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


{'currentNodeCount': 1,
 'targetNodeCount': 1,
 'nodeStateCounts': {'preparingNodeCount': 0,
  'runningNodeCount': 0,
  'idleNodeCount': 1,
  'unusableNodeCount': 0,
  'leavingNodeCount': 0,
  'preemptedNodeCount': 0},
 'allocationState': 'Steady',
 'allocationStateTransitionTime': '2024-02-17T10:45:21.420000+00:00',
 'errors': None,
 'creationTime': '2024-02-17T08:24:17.056201+00:00',
 'modifiedTime': '2024-02-17T08:24:26.830859+00:00',
 'provisioningState': 'Succeeded',
 'provisioningStateTransitionTime': None,
 'scaleSettings': {'minNodeCount': 1,
  'maxNodeCount': 8,
  'nodeIdleTimeBeforeScaleDown': 'PT1800S'},
 'vmPriority': 'Dedicated',
 'vmSize': 'STANDARD_D2_V2'}

## Hyperdrive Configuration

Here I will set configuration for different parameters for our hyperdrive run.

I will use `RandomParameterSampling` as the Sampling method, `BanditPolicy` as the Termination policy, and `SKLearn estimator` with the Primary metric as `AUC_weighted`.

In [5]:
env = Environment(name='hyper_env')

# Specify Python dependencies
env.python.conda_dependencies.add_pip_package('pandas')
env.python.conda_dependencies.add_pip_package('scikit-learn')

# Register the environment in your workspace
env.register(workspace=ws)

{
    "assetId": "azureml://locations/southcentralus/workspaces/4b9627bf-2c26-4525-823d-d4b80da461db/environments/hyper_env/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230509.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "hyper_env",
    "pyt

In [6]:
# Creating an early termination policy.
early_termination_policy = BanditPolicy(slack_factor=0.1,evaluation_interval=3)

#Creating the different params that will be used during training
param_sampling = RandomParameterSampling({"--criterion": choice("gini", "entropy"),
                                          "--bootstrap": choice("True", "False"), 
                                          "--max_depth": randint(10)})

#Creating estimator and hyperdrive config
src = ScriptRunConfig(source_directory=".", script="train.py",
                            compute_target=cpu_cluster,environment=env)

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                         policy=early_termination_policy, 
                                         primary_metric_name="AUC_weighted", 
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                         max_total_runs=8, 
                                         max_concurrent_runs=2, 
                                         run_config=src)

In [7]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

In the cell below, I use the `RunDetails` widget to show the different experiments.

In [8]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_4f718995-c51b-4a70-8126-6afe5d465c0c
Web View: https://ml.azure.com/runs/HD_4f718995-c51b-4a70-8126-6afe5d465c0c?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-253507/workspaces/quick-starts-ws-253507&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-02-17T11:10:13.228831][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space
[2024-02-17T11:10:13.6624043Z][SCHEDULER][INFO]Scheduling job, id='HD_4f718995-c51b-4a70-8126-6afe5d465c0c_0' 
[2024-02-17T11:10:13.7762138Z][SCHEDULER][INFO]Scheduling job, id='HD_4f718995-c51b-4a70-8126-6afe5d465c0c_1' 
[2024-02-17T11:10:13.723657][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.
[2024-02-17T11:10:14.1013737Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_4f718995-c51b-4a70-8126-6afe5d465c0c_0' 
[2024-02-17T11:10:14.1231988Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_4f7189

{'runId': 'HD_4f718995-c51b-4a70-8126-6afe5d465c0c',
 'target': 'automl-vs-hpyer',
 'status': 'Completed',
 'startTimeUtc': '2024-02-17T11:10:12.395129Z',
 'endTimeUtc': '2024-02-17T11:17:17.496927Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"AUC_weighted","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'b7fab8c8-22b3-44c4-a114-fd73b0525375',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_4f718995-c51b-4a70-8126-6afe5d465c0c_4',
  'score': '0.7440925087983912',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_4f718995-c51b-4a70-8126-6afe5d465c0c_4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telem

In [9]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

In the cell below, I get the best model from the hyperdrive experiments and display all the properties of the model.

In [10]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
 
print('Best Run Id: ', best_run.id)
print('AUC_weighted of Best Run is:', best_run_metrics['AUC_weighted'])
print('Parameter Values are:',best_run.get_details()['runDefinition']['arguments'])

Best Run Id:  HD_4f718995-c51b-4a70-8126-6afe5d465c0c_4
AUC_weighted of Best Run is: 0.7440925087983912
Parameter Values are: ['--bootstrap', 'False', '--criterion', 'entropy', '--max_depth', '8']


In [11]:
model = best_run.register_model(model_name='hyperdrive-model', 
                                model_path='outputs/model.joblib', 
                                tags={'Method':'Hyperdrive'}, 
                                properties={'AUC_weighted': best_run_metrics['AUC_weighted']})